###### References:  
- https://docs.python.org/3/reference/datamodel.html#objects-values-and-types
- Fluent Python by Luciano Ramalho. Chapter 8: Object References, Mutability, and Recycling


# Variables Are Not Boxes
Variables are labels, not boxes.

![variables](variables.png)

The Variable is assigned to an object, not the other way round.

In [1]:
class Gizmo:
    def __init__(self):
        print('Gizmo id: %d' % id(self))

x = Gizmo()

Gizmo id: 2442630765392


In [2]:
y = Gizmo() * 10

Gizmo id: 2442630765008


TypeError: unsupported operand type(s) for *: 'Gizmo' and 'int'

In [3]:
dir()

['Gizmo',
 'In',
 'Out',
 '_',
 '__',
 '___',
 '__builtin__',
 '__builtins__',
 '__doc__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 '_dh',
 '_i',
 '_i1',
 '_i2',
 '_i3',
 '_ih',
 '_ii',
 '_iii',
 '_oh',
 'exit',
 'get_ipython',
 'quit',
 'x']

# Identity, Equality, and Aliases

In [4]:
charles = {'name': 'Charles L. Dodgson', 'born': 1832}
lewis = charles
lewis is charles

True

In [5]:
id(charles), id(lewis)

(2442631748928, 2442631748928)

In [6]:
lewis['balance'] = 950
charles

{'name': 'Charles L. Dodgson', 'born': 1832, 'balance': 950}

In [7]:
alex = {'name': 'Charles L. Dodgson', 'born': 1832, 'balance': 950}

alex == charles

True

In [8]:
alex is not charles

True

![aliasing](aliasing.png)

# Choosing Between `==` and `is`

"*Every object has an identity, a type and a value. An object’s identity never changes once it has been created; you may think of it as the object’s address in memory. The `is` operator compares the identity of two objects; the id() function returns an integer representing its identity.*" 

--https://docs.python.org/3/reference/datamodel.html#objects-values-and-types

The `==` compares the value.

except when comparing to a singleton:

In [9]:
x is None

False

In [10]:
x is not None

True

The `is` operator is faster than `==`.

# Relative Immutability of Tuples

Immutability of tuples refers to the id and physical contents, not its referenced objects.

In [11]:
t1 = (1, 2, [30, 40])
t2 = (1, 2, [30, 40])
t1 == t2

True

In [12]:
id(t1[-1])

2442629959872

In [13]:
t1[-1].append(99)

t1

(1, 2, [30, 40, 99])

In [14]:
id(t1[-1])

2442629959872

In [15]:
t1 == t2

False

# Copies Are Shallow by Default

In [16]:
l1 = [3, [44, 55, 66], (7, 8, 9)]

l2 = list(l1)

l2

[3, [44, 55, 66], (7, 8, 9)]

In [17]:
l2 == l1

True

In [18]:
l2 is l1

False

In [19]:
l1.append(100)

print('l1:', l1)
print('l2:', l2)

l1: [3, [44, 55, 66], (7, 8, 9), 100]
l2: [3, [44, 55, 66], (7, 8, 9)]


In [20]:
l1[1].remove(55)

print('l1:', l1)
print('l2:', l2)

l1: [3, [44, 66], (7, 8, 9), 100]
l2: [3, [44, 66], (7, 8, 9)]


In [21]:
l2[1] += [33, 22]

print('l1:', l1)
print('l2:', l2)

l1: [3, [44, 66, 33, 22], (7, 8, 9), 100]
l2: [3, [44, 66, 33, 22], (7, 8, 9)]


In [22]:
l2[2] += (10, 11)

print('l1:', l1)
print('l2:', l2)

l1: [3, [44, 66, 33, 22], (7, 8, 9), 100]
l2: [3, [44, 66, 33, 22], (7, 8, 9, 10, 11)]


# Deep and Shallow Copies of Arbitrary Objects

In [23]:
class Bus:

    def __init__(self, passengers=None):
        if passengers is None:
            self.passengers = []
        else:
            self.passengers = list(passengers)

    def pick(self, name):
        self.passengers.append(name)

    def drop(self, name):
        self.passengers.remove(name)

In [24]:
import copy

In [25]:
bus1 = Bus(['Alice', 'Bill', 'Claire', 'David'])
bus2 = copy.copy(bus1)
bus3 = copy.deepcopy(bus1)
bus1.drop('Bill')
bus2.passengers

['Alice', 'Claire', 'David']

In [26]:
bus3.passengers

['Alice', 'Bill', 'Claire', 'David']

## Cyclic References

In [27]:
a = [10, 20]
b = [a, 30]

a.append(b)
a

[10, 20, [[...], 30]]

In [28]:
from copy import deepcopy

In [29]:
c = deepcopy(a)
c

[10, 20, [[...], 30]]

# Function Parameters as References
The only type of parameter passing in Python is *call by sharing*.

In [30]:
def f(a, b):
    a += b
    return a

x = 1
y = 2

f(x, y)

3

In [31]:
x, y

(1, 2)

In [32]:
a = [1, 2]
b = [3, 4]

f(a, b)

[1, 2, 3, 4]

In [33]:
a, b

([1, 2, 3, 4], [3, 4])

In [34]:
t = (10, 20)
u = (30, 40)

f(t, u)

(10, 20, 30, 40)

In [35]:
t, u

((10, 20), (30, 40))

# Mutable Types as Parameter Defaults: Bad Idea

In [36]:
class HauntedBus:
    """A bus model haunted by ghost passengers"""

    def __init__(self, passengers=[]): # parameter is bound to the default list object
        self.passengers = passengers  # makes self.passengers an alias for passengers, which is an alias of the default list.

    def pick(self, name):
        self.passengers.append(name)  # we are mutating the default list

    def drop(self, name):
        self.passengers.remove(name)

In [37]:
bus1 = HauntedBus(['Alice', 'Bill'])
bus1.passengers

['Alice', 'Bill']

In [38]:
bus1.pick('Charlie')
bus1.drop('Alice')
bus1.passengers

['Bill', 'Charlie']

In [39]:
bus2 = HauntedBus()
bus2.pick('Carrie')
bus2.passengers

['Carrie']

In [40]:
bus3 = HauntedBus()
bus3.passengers

['Carrie']

In [41]:
bus3.pick('Dave')
bus2.passengers

['Carrie', 'Dave']

In [42]:
dir(HauntedBus.__init__)

['__annotations__',
 '__call__',
 '__class__',
 '__closure__',
 '__code__',
 '__defaults__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__get__',
 '__getattribute__',
 '__globals__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__kwdefaults__',
 '__le__',
 '__lt__',
 '__module__',
 '__name__',
 '__ne__',
 '__new__',
 '__qualname__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']

In [43]:
HauntedBus.__init__.__defaults__

(['Carrie', 'Dave'],)

In [44]:
HauntedBus.__init__.__defaults__[0] is bus2.passengers

True

# Defensive Programming with Mutable Parameters
Need to align the expectations of the coder and the caller.

In [45]:
class TwilightBus:
    """A bus model that makes passengers vanish"""

    def __init__(self, passengers=None):
        if passengers is None:
            self.passengers = []  # create new empty list
        else:
            self.passengers = passengers  # this makes self.passengers an alias for the argument passed

    def pick(self, name):
        self.passengers.append(name)

    def drop(self, name):
        self.passengers.remove(name) 

In [46]:
basketball_team = ['Sue', 'Tina', 'Maya', 'Diana', 'Pat']

In [47]:
bus = TwilightBus(basketball_team)

In [48]:
bus.drop('Tina')
bus.drop('Pat')
basketball_team

['Sue', 'Maya', 'Diana']

# `del` and Garbage Collection
"*Objects are never explicitly destroyed; however, when they become unreachable they may be garbage-collected.*" 

    --https://docs.python.org/3/reference/datamodel.html
    
The `del` statement deletes names, not objects.

In CPython, garbage collection keeps count of hoe many references point to each object, and destroys it once it reaches zero.

In [49]:
import weakref

In [50]:
s1 = {1, 2, 3}
s2 = s1

def bye():
    print('Gone with the wind...')
    
ender = weakref.finalize(s1, bye) # Return a callable finalizer object which will be called when obj is garbage collected.
ender.alive

True

In [51]:
del s1
ender.alive

True

In [52]:
s2 = 'spam'

Gone with the wind...


In [53]:
ender.alive

False

# Weak References
Commonly used in caching.

Weak references do not increase an object's reference count.

An object that is the target of a reference is called the referent.

In [54]:
a_set = {0, 1}
wref = weakref.ref(a_set)
wref

<weakref at 0x00000238B8339680; to 'set' at 0x00000238B81E9BA0>

Returns the referenced object:

In [55]:
wref()

{0, 1}

In [56]:
a_set = {2, 3, 4}

In [57]:
wref()

{0, 1}

In [58]:
wref() is None

False

In [59]:
wref() is None

False

# The WeakValueDictionary Skit
Implements a mutable mapping where values are weak references to objects. When a referred object is garbage collected, the corresponding key is automatically removed from the `WeakValueDictionary`.

In [60]:
class Cheese:

    def __init__(self, kind):
        self.kind = kind

    def __repr__(self):
        return 'Cheese(%r)' % self.kind

In [61]:
stock = weakref.WeakValueDictionary()

catalog = [Cheese('Red Leicester'), Cheese('Tilsit'), Cheese('Brie'), Cheese('Parmesan')]

for cheese in catalog:
    stock[cheese.kind] = cheese

sorted(stock.keys())

['Brie', 'Parmesan', 'Red Leicester', 'Tilsit']

In [62]:
del catalog

sorted(stock.keys())

['Parmesan']

In [63]:
sorted(stock.keys())

['Parmesan']

## Limitations of Weak References

In [64]:
class MyList(list):
    """list subclass whose instances may be weakly referenced"""

In [65]:
a_list = MyList(range(10))

`a_list` can be the target of a weak reference

In [66]:
wref_to_a_list = weakref.ref(a_list)

# Tricks Python Plays with Immutables

In [67]:
t1 = (1, 2, 3)
t2 = tuple(t1)
t2 is t1

True

In [68]:
t3 = t1[:]
t3 is t1

True

In [69]:
fs1 = frozenset([4,5,6])
fs2 = fs1.copy()
fs2 is fs1

True

In [70]:
t3 = (1, 2, 3)
t3 is t1

False

## *Interning*: Sharing of string literals

In [71]:
s1 = 'ABC'
s2 = 'ABC'

s2 is s1

True